# 智能客服监控 Agent - 任务详解

## 📋 任务概述

这是一个**全栈开发+运维自动化**的实操项目，要求你设计并实现一个**具备感知能力的智能客服系统**，它能够：

1. ✅ 基于知识库回答业务问题
2. ✅ 实时感知系统健康状态
3. ✅ 自动触发运维操作（告警、记录）

---

## 🎯 核心能力要求

### 1. **智能问答能力**
- 基于提供的知识库文档回答用户问题
- 严禁"幻觉"，没有答案时要诚实回答
- 能识别用户是否在询问系统稳定性

### 2. **系统感知能力**
- 实时监控 API 状态（通过 Uptime Kuma 或模拟）
- 能区分 200 OK 和 500 Error 等不同状态
- 能读取并理解监控日志

### 3. **自动化执行能力**
- 当 API 异常时，自动触发飞书 Webhook 告警
- 自动生成 Apifox 故障文档记录
- 形成"监控→告警→文档"的自动化闭环

---

## 📥 输入数据格式

系统会读取 `inputs.json` 文件，包含多个测试用例：

```json
[
  {
    "case_id": "C001",
    "user_query": "你们平台的计费模式是怎样的？",
    "api_status": "200 OK",
    "api_response_time": "120ms",
    "monitor_log": []
  }
]
```

### 字段说明：
- **case_id**: 测试用例的唯一标识
- **user_query**: 用户的提问内容
- **api_status**: API 当前状态（如 "200 OK", "500 Internal Server Error"）
- **api_response_time**: API 响应时间
- **monitor_log**: 监控日志列表，包含历史错误记录

---

## 📤 输出数据格式

需要生成 `outputs/results.json` 文件：

```json
[
  {
    "case_id": "C001",
    "reply": "根据平台文档，我们提供按量付费和包月订阅两种模式...",
    "action_triggered": null
  },
  {
    "case_id": "C002",
    "reply": "非常抱歉，检测到模型 API 在 10:00:01 出现了短暂的连接异常...",
    "action_triggered": {
        "feishu_webhook": "Sent success",
        "apifox_doc_id": "DOC_20241212_ERROR_01"
    }
  }
]
```

### 字段说明：
- **case_id**: 对应输入的测试用例ID
- **reply**: 给用户看的智能回复
- **action_triggered**: 触发的系统动作（正常时为 null，异常时记录具体操作）

---

## ⚙️ 核心逻辑流程

```
用户提问 + API状态 → Agent分析 → 生成回复 + 触发动作

1. 读取输入数据
   ↓
2. 判断API状态
   ├─ 200 OK: 系统正常
   │   ├─ 如果是稳定性询问 → 基于监控日志回答
   │   └─ 如果是业务问题 → 基于知识库回答
   │
   └─ 非200: 系统异常
       ├─ 生成故障回复（告知用户真实情况）
       ├─ 触发飞书Webhook告警
       └─ 创建Apifox故障文档
   ↓
3. 生成输出结果
```

---

## 🚨 关键规范

### ❌ 严禁的行为：

1. **禁止"睁眼说瞎话"**
   - 当 API 报错时，不能说"系统很稳定"
   - 必须诚实告知用户真实情况

2. **禁止"幻觉"**
   - 知识库中没有的答案，不能说"根据文档..."
   - 必须回答"知识库中未找到相关信息"

3. **禁止"不作为"**
   - 检测到异常时，必须触发告警和文档记录
   - 不能只回复用户而不执行运维操作

---

## 💡 实现建议

### 技术栈选择：
- **Python**: 适合快速开发，有丰富的HTTP库
- **Node.js**: 如果需要更强的异步处理能力

### 关键库推荐：
- `requests`: 调用Webhook和API
- `json`: 处理数据格式
- `datetime`: 生成时间戳

### 代码结构建议：

```
smart_agent/
├── main.py              # 主程序入口
├── agent.py             # Agent核心逻辑
├── knowledge_base.py    # 知识库查询
├── monitor.py           # 监控状态判断
├── actions.py           # 动作执行（飞书、Apifox）
├── inputs.json          # 输入文件
└── outputs/
    └── results.json     # 输出文件
```

---

## 🎓 考察点总结

这个任务综合考察了以下能力：

| 能力类别 | 具体考察点 |
|---------|----------|
| **文档理解** | 能否准确理解任务要求和数据格式 |
| **逻辑设计** | 能否设计清晰的判断和执行流程 |
| **编码实现** | 能否将逻辑转化为可运行的代码 |
| **运维思维** | 是否理解监控、告警、文档的重要性 |
| **诚实性** | 能否避免"幻觉"，诚实面对系统状态 |

---

## 🚀 下一步行动

1. **理解任务**: 你已经完成了 ✓
2. **设计系统**: 规划代码结构
3. **编写代码**: 实现核心逻辑
4. **测试验证**: 用示例数据测试
5. **提交成果**: 生成最终结果文件

---

## 💬 总结

这是一个**非常实用**的项目，在真实的AI平台运维中，这种"能感知、会告警、说实话"的智能客服是非常有价值的。

它不仅解决了用户"不知道系统出问题"的痛点，还实现了从发现问题到解决问题的自动化流程，体现了 **RAG + DevOps** 的结合。

现在开始动手实现吧！🎯


# 定义常量

In [ ]:
import pickle

from jupyter_core.paths import jupyter_data_dir
from langchain.chains.question_answering.map_reduce_prompt import messages

with open("../api_key.pkl", "rb") as f:
   API_KEY=pickle.load(f)

# print(API_KEY)

MODEL_ID= "ali/qwen3-omni-flash"

import csv
import datetime

def log(s: str) -> None:
    """
    把一条日志追加到 log.csv。
    第一列：当前时间（ISO 8601，带毫秒）
    第二列：s 本身（自动处理里面的逗号、引号、换行）
    文件不存在会自动创建，存在则追加。
    """
    # 构造一行
    now=datetime.datetime.now()
    row = [f"{now.year%100}{now.month:02}{now.day:02} {now.hour:02}{now.minute:02}{now.second:02}.{now.microsecond/1000:03.0f}", s]

    # 以追加模式打开，newline='' 防止 Windows 多空行
    with open('log.csv', 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_MINIMAL)
        # 如果文件刚创建，可写表头（按需）
        if f.tell() == 0:
            writer.writerow(['timestamp', 'message'])
        writer.writerow(row)


import http.client
import json
import time

conn = http.client.HTTPSConnection("router.shengsuanyun.com")
HEADERS = {
   'HTTP-Referer': 'https://www.postman.com',
   'X-Title': 'Postman',
   'Authorization': API_KEY,
   'Content-Type': 'application/json'
}


KNOWLEDGE_BASE = {
    "计费模式": "根据平台文档，我们提供按量付费和包月订阅两种模式。按量付费的价格为每千次调用 0.1 元，包月订阅为 299 元/月不限量调用。",
    "系统稳定性": "系统稳定性信息需要结合实时监控数据来回答。"
}

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "飞书告警",
            "description": "通过 Webhook 向指定飞书账号/群发送富文本卡片。",
            "parameters": {
                "type": "object",
                "properties": {
                    "报错时间": {
                        "type": "string",
                        "description": "报错时间",
                    },
                    "错误代码": {
                        "type": "string",
                        "description": "错误代码",
                    },
                    "当前延迟": {
                        "type": "string",
                        "description": "当前延迟",
                    },

                },
                "required": ["报错时间","错误代码","当前延迟"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "Apifox 文档同步",
            "description": "调用 Apifox 开放 API，自动生成一篇新的接口文档或错误日志。",
            "parameters": {
                "type": "object",
                "properties": {
                    "标题": {
                        "type": "string",
                        "description": "文档标题格式：[故障记录] YYYY-MM-DD HH:mm:ss",
                    },
                    "内容": {
                        "type": "string",
                        "description": "文档内容",
                    }
                },
                "required": ["标题","内容"],
            },
        },
    },
]

# 胜算云API调用

In [ ]:
# payload = json.dumps({
#    "model": MODEL_ID,
#    "stream": True,
#    "messages": [
#       {
#          "role": "user",
#          "content": "讲个笑话"
#       },
#       {
#          "role": "assistant",
#          "content": "cong qian you zuo shan"
#       },
#       {
#          "role": "user",
#          "content": "继续"
#       }
#    ],
#    "stream_options": {
#       "include_usage": True
#    }
# })
# start_time = time.time()
#conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
# elapsed_time = time.time()-start_time
# print(f"executed in {elapsed_time:.4f} seconds")
# res = conn.getresponse()
#
# for line in res:
#     line = line.decode('utf-8').strip()
#     try:
#         json_str = line[6:]
#         obj = json.loads(json_str)
#         content = obj["choices"][0]["delta"]["content"]
#         if content != None:
#             print(content,end="")
#         if "usage" in obj:
#             print("\n\nusage:",obj["usage"])
#     except:
#         pass
#     if line != "":
#         log(line)


In [ ]:
# message=[
#       {
#          "role": "system",
#          "content": """你是一个“具备感知能力”的智能客服。你能
# 1.基于文档回答业务问题
# 2.实时连接 Uptime Kuma 监控系统
# 3.当 API 报错时，自动通过 Webhook 通知飞书运维群，并在Apifox 中记录故障文档
# 4.当用户询问“今天系统稳定吗”时，你能基于监控历史给出真实的回答，而不是瞎编。"""
#       },
#       {
#          "role": "user",
#          "content": "今天系统稳定吗"
#       },
#    ]

def LLM_invoke(message):
    payload = json.dumps({
       "model": MODEL_ID,
       "stream": False,
       "messages": message,
       "stream_options": {
          "include_usage": True
       }
    })
    start_time = time.time()

    conn.request("POST", "/api/v1/chat/completions", payload, HEADERS)
    res = conn.getresponse()
    obj = json.loads(res.read().decode('utf-8'))
    elapsed_time = time.time()-start_time
    try:
        content=obj["choices"][0]["message"]["content"]
        print("-"*100,f"\nexecuted in {elapsed_time:.4f} seconds")
        print("usage:",obj["usage"])
    except:
        print("msg=",message)
        print("obj=",obj)
        content="（比赛无关）胜算云API错误"
    log(obj)
    return content

# print(LLM_invoke(message))

In [ ]:
del conn

# Agent构建

In [ ]:
"""
智能客服监控 Agent – LangGraph 版
"""
import json, os, time, requests
import datetime
from typing import Dict, List, Optional, Any

from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from pydantic import BaseModel, Field

# --------------------------------------------------
# 1. 知识库 / 监控 / 动作 工具函数
# --------------------------------------------------
class KnowledgeBase:
    def __init__(self):
        self.knowledge = {
            "计费": "按量付费每千次 0.1 元，包月 299 元不限量。",
            "价格": "按量付费每千次 0.1 元，包月 299 元不限量。",
            "费用": "按量付费每千次 0.1 元，包月 299 元不限量。",
            "功能": "平台支持文本生成、图像识别、语音转换等多种AI能力。",
            "API": "提供 RESTful 接口，支持多语言 SDK。",
            "文档": "官方文档中心：https://docs.example.com",
        }

    def query(self, question: str) -> str:
        q = question.lower()
        for k, v in self.knowledge.items():
            if k.lower() in q:
                return v
        return "知识库中未找到相关信息"


class Monitor:
    @staticmethod
    def is_healthy(api_status: str) -> bool:
        return api_status == "200 OK"

    @staticmethod
    def get_stability_info(monitor_log: List[Dict]) -> str:
        if not monitor_log:
            return "系统目前运行稳定"
        latest = monitor_log[-1]
        return f"系统在 {latest['timestamp']} 曾出现 {latest['msg']}，目前已恢复。"


class Actions:
    @staticmethod
    def send_feishu_alert(api_status: str, response_time: str, monitor_log: List[Dict]) -> str:
        print("📤 发送飞书告警（模拟）")
        return "Sent success"

    @staticmethod
    def create_apifox_doc(api_status: str, response_time: str, monitor_log: List[Dict]) -> str:
        print("📄 创建 Apifox 故障文档（模拟）")
        return f"DOC_{datetime.datetime.now():%Y%m%d_%H%M%S}_ERROR"


# --------------------------------------------------
# 2. LangGraph 状态定义
# --------------------------------------------------
class AgentState(BaseModel):
    case_id: str
    user_query: str
    api_status: str
    api_response_time: str
    monitor_log: List[Dict]

    # 运行过程中产生的中间数据
    user_intent: Optional[str] = None
    system_healthy: Optional[bool] = None
    knowledge_snippet: Optional[str] = None
    final_reply: Optional[str] = None
    action_apifox_id: Optional[str] = None


# --------------------------------------------------
# 3. 节点函数
# --------------------------------------------------
kb = KnowledgeBase()
monitor = Monitor()
actions = Actions()

def print_state_info(func):
    def inner(*args):
        print(func.__name__,"entered:",args)
        log(*args)
        func(*args)
    return inner

@print_state_info
def node_retrieve(state: AgentState) -> AgentState:
    """检索知识库 + 判断系统健康"""
    """让大模型决定回复内容（系统正常分支）"""
    sys_msg = (
        "你是智能客服助手。请根据用户问题判断用户的意图，如果用户问的是业务问题，输出\"业务\"，如果用户问的是系统状态问题，输出\"状态\"。"
    )
    user_msg = f"{state.user_query}"
    messages = [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": user_msg},
    ]
    judgement =  LLM_invoke(messages)
    print("judgement:",judgement)
    if judgement =="业务":
        state.user_intent="业务"
    elif judgement =="状态":
        state.user_intent="状态"
    else:
        state.user_intent=judgement


    print("state.user_intent:",state.user_intent)
    state.system_healthy = monitor.is_healthy(state.api_status)
    state.knowledge_snippet = kb.query(state.user_query)
    return {"user_intent":judgement}

@print_state_info
def node_knowledge(state: AgentState) -> AgentState:
    """让大模型决定回复内容（系统正常分支）"""
    sys_msg = (
        "你是智能客服助手。请根据【用户问题】和【知识库片段】生成一段自然、简洁、口语化的回复。"
        "如果知识库片段为空，可委婉表示暂未找到信息。"
    )
    user_msg = f"【用户问题】{state.user_query}\n【知识库片段】{state.knowledge_snippet}"
    messages = [
        {"role": "system", "content": sys_msg},
        {"role": "user", "content": user_msg},
    ]
    state.final_reply =  LLM_invoke(messages)
    return state

# @print_state_info
# def node_llm_decide(state: AgentState) -> AgentState:
#     """让大模型决定回复内容（系统正常分支）"""
#     sys_msg = (
#         "你是智能客服助手。请根据【用户问题】和【知识库片段】生成一段自然、简洁、口语化的回复。"
#         "如果知识库片段为空，可委婉表示暂未找到信息。"
#     )
#     user_msg = f"【用户问题】{state.user_query}\n【知识库片段】{state.knowledge_snippet}"
#     messages = [
#         {"role": "system", "content": sys_msg},
#         {"role": "user", "content": user_msg},
#     ]
#     state.final_reply =  LLM_invoke(messages, MODEL_ID)
#     return state
#
#
# @print_state_info
# def node_alert(state: AgentState) -> AgentState:
#     """触发飞书告警"""
#     actions.send_feishu_alert(state.api_status, state.api_response_time, state.monitor_log)
#     return state
#
# @print_state_info
# def node_doc(state: AgentState) -> AgentState:
#     """创建 Apifox 文档"""
#     state.action_apifox_id = actions.create_apifox_doc(
#         state.api_status, state.api_response_time, state.monitor_log
#     )
#     return state

@print_state_info
def node_dummy(state: AgentState) -> AgentState:
    """dummy"""
    pass
    return state

# --------------------------------------------------
# 4. 构建图
# --------------------------------------------------
def build_graph():
    workflow = StateGraph(AgentState)

    workflow.add_node("retrieve", node_retrieve)
    workflow.add_node("knowledge", node_knowledge)
    # workflow.add_node("llm_decide", node_llm_decide)
    # workflow.add_node("alert", node_alert)
    # workflow.add_node("doc", node_doc)

    workflow.add_node("server", node_dummy)

    workflow.set_entry_point("retrieve")

    # 条件边：retrieve 之后根据系统状态分支
    @print_state_info
    def _router(state: AgentState):
        if state.user_intent=="状态":
            return "状态"
        elif state.user_intent=="业务":
            return "业务"

    workflow.add_conditional_edges(
        "retrieve",
        _router,
        {"状态": "server", "业务": "knowledge"},
    )

    # 状态分支：alert -> doc -> error_reply -> END
    workflow.add_edge("server", END)

    # 业务分支：knowledge -> END
    workflow.add_edge("knowledge", END)

    memory = MemorySaver()
    graph = workflow.compile(checkpointer=False)#memory) 暂不使用检查点
    mmd_graph=graph.get_graph().draw_mermaid()
    with open("graph.mmd", "w") as f:
        f.write(mmd_graph)
    log(mmd_graph)
    return graph


# --------------------------------------------------
# 5. 对外暴露的 Agent 类
# --------------------------------------------------
class SmartAgent:
    def __init__(self):
        self.graph = build_graph()

    def process(self, case: Dict[str, Any]) -> Dict[str, Any]:
        state = AgentState(**case)
        print("="*100,"\ncase:", case)
        thread = {"configurable": {"thread_id": case["case_id"]}}
        print("-"*100,"\nstate.model_dump():",state.model_dump())
        final_state = self.graph.invoke(state.model_dump())#, config=thread)
        print("-"*100,"\nagent response:", final_state["final_reply"])
        # 组装成旧格式
        return {
            "case_id": final_state["case_id"],
            "reply": final_state["final_reply"],
            "action_triggered": (
                {"apifox_doc_id": final_state["action_apifox_id"]}
                if final_state["action_apifox_id"]
                else None
            ),
        }




# 文件输入

In [ ]:
# 读取输入数据
try:
    with open("../inputs.json", "r", encoding="utf-8") as f:
        inputs = json.load(f)
    print(f"✅ 成功读取 {len(inputs)} 个测试用例")
except FileNotFoundError:
    print("❌ 未找到 inputs.json，使用内置测试数据")
    inputs = []
    """
        {
            "case_id": "C001",
            "user_query": "你们平台的计费模式是怎样的？",
            "api_status": "200 OK",
            "api_response_time": "120ms",
            "monitor_log": [],
        },
        {
            "case_id": "C002",
            "user_query": "刚才模型是不是挂了？怎么一直没反应？",
            "api_status": "500 Internal Server Error",
            "api_response_time": "Timeout",
            "monitor_log": [
                {"timestamp": "10:00:01", "status": "Error", "msg": "Connection Refused"}
            ],
        },
        {
            "case_id": "C003",
            "user_query": "今天系统稳定吗？",
            "api_status": "200 OK",
            "api_response_time": "150ms",
            "monitor_log": [
                {"timestamp": "09:30:15", "status": "Error", "msg": "Timeout"},
                {"timestamp": "09:35:20", "status": "OK", "msg": "Recovered"},
            ],
        },
    ]"""


# 运行Agent

In [ ]:
print("=" * 100)
print("智能客服监控 Agent启动")
print("=" * 100)

agent = SmartAgent()
results = agent.process(inputs[0])
# results = [agent.process(case) for case in inputs]

# 文件输出

In [ ]:
os.makedirs("outputs", exist_ok=True)
with open("../outputs/results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("\n" + "=" * 60)
print("✅ 处理完成！结果已保存到 outputs/results.json")
print("=" * 60 + "\n")

print("【处理摘要】")
for r in results:
    print(f"  {r['case_id']}: 已回复 {'✓' if r['action_triggered'] else '✗'} 触发动作")